# Making trisyllables from a csv file and .wavs of individual syllables
trisyllables are in snake case, with prominence and weight denoted as follows: 
NB: this works because python's string default is case sensitive.

       L, unstressed: cv
       L, prominent: cV
       H, unstressed: cvv
       H, prominent: cAA

for example:  

       QI: (LLL) 
              target pattern: learning 
              left edge: bA_ba_ba 
              right edge: ba_ba_bA

decision pairs: target + two incorrect options of possible positions of prominence: 

       dóloze, dolóze, dolozé
       * dO_lo_ze, do_lO_ze, dolozE
       each edge group has three: correct, incorrect1, incorrect2
              
       QS: 
       HLL: bAA_ba_ba 
       LHL: ba_bAA_ba
       H'HL: baa_bAA_ba

In [ ]:
import pandas as pd

stim_input = "/Users/sarah/Git/stress_learn/syllables/"
trisyllables_df = pd.read_csv(stim_input + "stim.csv")
trisyllables_df.head()

In [ ]:

left = []
mid = []
right = []
#store the word in this row
for item in trisyllables_df.nonce: 
    nonce = item
    l_index = 1
    m = 4
    r_index = len(nonce)
    tmpLeft = nonce[0] + nonce[l_index].upper() + nonce[2:r_index]
    tmpMid = nonce[0:(m)] + nonce[m].upper() + nonce[(m + 1):r_index]
    tmpRight = nonce[0:r_index-1] + nonce[r_index-1].upper()

    left.append(tmpLeft)
    mid.append(tmpMid)
    right.append(tmpRight)
#print(left, mid, right)
trisyllables_df["left"] = left
trisyllables_df["mid"] = mid
trisyllables_df["right"] = right
trisyllables_df.head()


In [ ]:
qi_conditions_df = trisyllables_df[["left","mid","right"]]
print(qi_conditions_df)

# print("AFTER CONVERSION\n", after, "\n")

In [ ]:
import parselmouth
#set the directory for the .wav files of individual syllables
syll_root = "/Users/sarah/Git/stress_learn/syllables/"
#set the directory for new .wav files to be saved after concatenation
output_words = "/Users/sarah/Git/stress_learn/words/"


def affixer(trisyllable,output):
       #split string into individual syllables

    triplet = (trisyllable.split('_'))
    #print(triplet)
    init = triplet[0]
    penult = triplet[1]
    ultima = triplet[2]
    #print(init, penult, ultima)
    #create sound object of each syllable
    s1 = syll_root+"short_soft/" + init + ".wav"
    s2 = syll_root+ "short_soft/" + penult + ".wav"
    s3 = syll_root+ "short_stress/" + ultima + ".wav"
    syll1 = parselmouth.Sound(s1)
    syll2 = parselmouth.Sound(s2)
    syll3 = parselmouth.Sound(s3)
    #create list of sound objects
    tri_syll = [syll1,syll2,syll3]
    #tmpword = parselmouth.Sound.concatenate(syll1,syll2,0.0)
    word = parselmouth.Sound.concatenate(tri_syll,0.0)
    word.save((output + trisyllable+ ".wav"),parselmouth.SoundFileFormat.WAV)

for item in qi_conditions_df["right"]:
    affixer(item, output_words)